# Building your own recommender system

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

#### Read the data

In [2]:
PATH = 'google_books_1299.csv'

def get_data(path_to_data):

    data = pd.read_csv(f'{path_to_data}',index_col=0)
    data["title"] = data["title"].str.lower()
    data = data.dropna()
    data.index = [i for i in range(0,len(data))]
    return data

In [3]:
data = get_data(PATH)

# Create a content-based recommender system using cosine similarity

In [4]:
data['generes'] = data['generes'].astype('string')

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/Users/su/opt/anaconda3/envs/project/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def recommender(genres, data):
    book_dic = {}
    
    for index, value in data.iterrows():
        genres_embeddings = model.encode([genres])
        value_embeddings = model.encode([value[9]])
        cosine = 1 - spatial.distance.cosine(genres_embeddings, value_embeddings)
        if cosine > 0.7:
            book_dic[index] = cosine
        else:
            continue
    if book_dic and len(list(book_dic.keys()))>=5:
        book_dic = dict(sorted(book_dic.items(), key=lambda x: x[1], reverse=True))
        book_indexs = list(book_dic.keys())[:5]
        recommendation = pd.DataFrame(columns=['ISBN','title'])
        for book_index in book_indexs:
            book_ISBN = data['ISBN'].iloc[book_index]
            book_title = data['title'].iloc[book_index]
            recommendation=recommendation.append({'ISBN' : book_ISBN , 'title' : book_title} , ignore_index=True)      
        return recommendation
    elif book_dic:
        print("We can only give you less than 5 recommendations")
        book_dic = dict(sorted(book_dic.items(), key=lambda x: x[1], reverse=True))
        book_indexs = list(book_dic.keys())
        recommendation = pd.DataFrame(columns=['ISBN','title'])
        for book_index in book_indexs:
            book_ISBN = data['ISBN'].iloc[book_index]
            book_title = data['title'].iloc[book_index]
            recommendation=recommendation.append({'ISBN' : book_ISBN , 'title' : book_title} , ignore_index=True)      
        return recommendation
        
    else:
        recommendation = "Sorry, we do not have any recommentation for you."
        return recommendation
        
def Content_based_recommender(data):

    data = data[data['generes'].notna()]
    data['generes'] = data['generes'].str.lower()

    print(f"What type of genre do you like? \n\nYou can choose from the following:\n\n{set(data['generes'])}")
    genres = input().lower()
    
    recommendations = recommender(genres, data)


    return recommendations

In [8]:
Content_based_recommender(data)

What type of genre do you like? 

You can choose from the following:

{'fiction , mystery &amp, detective , collections &amp, anthologies', 'comics & graphic novels , horror', 'fiction , fantasy , epic', 'biography &amp, autobiography , military', 'fiction , thrillers , political', 'business &amp, economics , industries , general', 'young adult fiction , animals , mythical creatures', 'business &amp, economics , personal finance , general', 'psychology , cognitive psychology &amp, cognition', 'fiction , mystery &amp, detective , international mystery &amp, crime', 'fiction , fantasy , military', 'business &amp, economics , accounting , financial', 'fiction , fairy tales, folk tales, legends &amp, mythology', 'comics & graphic novels , science fiction', 'art , popular culture', 'business &amp, economics , development , business development', 'fiction , thrillers , psychological', 'political science , civics &amp, citizenship', 'games &amp, activities , video &amp, electronic', 'performi

fiction,history


ISBN                                              title
0  9780007423309       prince of thorns (the broken empire, book 1)
1  9781476789477  war and peace: with bonus material from give w...
2  9780007422135                           and then there were none
3  9780007422586           the mysterious affair at styles (poirot)
4  9780007463497                    the mysterious affair at styles

In [ ]:
Content_based_recommender(data)